In [1]:
import torch
import torchvision
from torchvision.io import read_image
from torch.utils.data import DataLoader
import torch.nn as nn

import os
from os import listdir
from os.path import isfile, join

import csv

from rcnn_dataset import RCNNDataset

from timeit import default_timer as timer

In [10]:
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

print(device)

cuda


In [ ]:
# #load image list for input
# filelist = [f for f in listdir(dir_path) if isfile(join(dir_path, f)) and f!='annotations.csv']
# print(len(filelist))

# images = []

# #load images
# for f in filelist:
#     temp_path = join(dir_path, f)
#     img = read_image(temp_path)
#     img = img.float().to(device)

#     images.append(img)

In [ ]:
# #load data from annotations file to get target
# targets = []
# with open('..\\CNN_test_sandbox\\Dataset\\Local (Training)\\annotations.csv', 'r') as file:
#     reader = csv.reader(file)
#     rows = []
#     for row in reader:
#         rows.append(row)

#     for f in filelist:
#         boxes = []
#         labels = []
#         for i in range(len(rows)):
#             if rows[i][0] == f:
#                 box = [
#                     float(rows[i][2]),
#                     float(rows[i][3]),
#                     float(rows[i][4]),
#                     float(rows[i][5])
#                 ]
                
#                 label = int(rows[i][1])
#                 boxes.append(box)
#                 labels.append(label)
#         boxes = torch.tensor(boxes, device=device)
#         labels = torch.tensor(labels, device=device)
#         temp_entry = {}
#         temp_entry['boxes'] = boxes
#         temp_entry['labels'] = labels
#         targets.append(temp_entry)
                

In [19]:
def collater_function(batch):
    final_targets = []
    new_images = []
    for i in range(len(batch)):
        #sample is a tuple of an image and its targets
        #unpack
        targets = batch[i][1]
        image = batch[i][0]

        new_dict_entry = {}
        #process targets
        new_dict_entry['boxes'] = torch.tensor(targets['boxes'], device=device)
        new_dict_entry['labels'] = torch.tensor(targets['labels'], device=device)

        final_targets.append(new_dict_entry)

        image = image.to(device)
        new_images.append(image)

    return (new_images, final_targets)

In [20]:
batch_size = 2

training_dataset = RCNNDataset(
    annotations='D:\\GitHub\\Hindi-Handwriting-Recognition\\CNN_test_sandbox\\Dataset\\Local (Training)\\annotations.csv',
    img_dir='D:\\GitHub\\Hindi-Handwriting-Recognition\\CNN_test_sandbox\\Dataset\\Local (Training)',
    device=device
)

training_loader = DataLoader(training_dataset, batch_size=batch_size, shuffle=True, collate_fn=collater_function)

testing_dataset = RCNNDataset(
    annotations='D:\\GitHub\\Hindi-Handwriting-Recognition\\CNN_test_sandbox\\Dataset\\Local (Testing)\\annotations.csv',
    img_dir='D:\\GitHub\\Hindi-Handwriting-Recognition\\CNN_test_sandbox\\Dataset\\Local (Testing)',
    device=device
)

testing_loader = DataLoader(testing_dataset, batch_size=batch_size, shuffle=True, collate_fn=collater_function)

Dataset Loaded
Dataset Loaded


In [21]:
for images, targets in training_loader:
    print(images)

[tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]], device='cuda:0'), tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0.,

KeyboardInterrupt: 

In [22]:
def test_and_evaluate(model, device, optimizer, scheduler, epochs, batch_size, training_loader, testing_loader, testing_dataset):
    n_total_steps = len(training_loader)
    print('Training')
    model.train()
    for epoch in range(epochs):
        start = timer()
        for i, (images, targets) in enumerate(training_loader):
            #print(targets)
            # images = images.to(device)
            # targets = targets.to(device)
    
            loss_dict = model(images, targets)
            loss_accumulated = sum(loss for loss in loss_dict.values())
            
            optimizer.zero_grad()
            loss_accumulated.backward()
            optimizer.step()

        scheduler.step()
        end = timer()
        print(f'Time for Epoch {epoch+1}: {end-start:.4f} seconds')
    
    #Evaluate
    print('Evaluating')
    model.eval()
    sample_limit = int(testing_dataset.__len__() / 47)
    with torch.no_grad():
        n_correct = 0
        n_samples = 0
        n_class_correct = [0 for i in range(47)]
        n_class_samples = [0 for i in range(47)]
        for images, labels in testing_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)

            _, predicted = torch.max(outputs, 1)
            n_samples += labels.size(0)
            n_correct += (predicted == labels).sum().item()

            test_limit = min(batch_size, sample_limit)

            for i in range(test_limit):
                label = labels[i]
                pred = predicted[i]
                label = int(label)
                if (label == pred):
                    n_class_correct[label] += 1
                n_class_samples[label] += 1

        acc = 100.0 * n_correct / n_samples
        print(f'Accurracy of the network: {acc}%')

        return acc

In [23]:
#rcnn test
learning_rate = 0.05
for num_epochs in range(10,51, 10):
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(progress=True, num_classes=47)
    model.to(device)

    optimizer = torch.optim.SGD(model.parameters(), lr  = learning_rate)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs, eta_min=0.001)

    acc = test_and_evaluate(model, device, optimizer, scheduler, num_epochs, batch_size, training_loader, testing_loader, testing_dataset)
    

    

Training


KeyboardInterrupt: 